<a href="https://colab.research.google.com/github/profteachkids/subtitle_generator/blob/main/subtitle_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter, ImageOps
import requests
from io import BytesIO
from google.colab import files

colors={0:(255, 255, 255, 255),
    1:(0, 0, 0, 255),
    2:(255, 0, 0, 255),
    3:(255, 0, 221, 255),  #pink
    4:(0,0,255,255), #blue
    5:(0,255,0.69,255),
    6:(128,0,128,255)} #purple

text="""after the questionable
016events of the last part
014where Spadeless lost
his pets""" 

root='./subtitles/'
if not os.path.exists(root):
    os.makedirs(root)
texts = text.split('\n')

imw,imh=1920,250

req = requests.get("https://github.com/google/fonts/blob/main/ofl/dosis/static/Dosis-Bold.ttf?raw=true")
fnt = ImageFont.truetype(BytesIO(req.content), 115)

i=0
w_start=0
for text in texts:
    z = re.match(r"[0-9]{3}", text)
    if z:
        font_c = int(text[0])
        outline_c = int(text[1])
        shadow_c = int(text[2])
        joined = ' '.join(text[3:].split())
    else:
        font_c=0
        outline_c=1
        shadow_c=1
        joined= ' '.join(text.split())

    w_end = w_start + len(text.split())
    name = root+f'a_{i:04d}_{w_start:04d}_{w_end:04d}.png'
    i+=1
    w_start = w_end


    im = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d = ImageDraw.Draw(im)
    w,h=d.textsize(joined,fnt)
    d.text(((imw-w)//2,(imh-h)//2), joined, font=fnt, fill=colors[shadow_c])
    im = im.filter(ImageFilter.GaussianBlur(radius=30))
    imnp_raw=np.asarray(im)
    imnp=imnp_raw.copy()
    imnp[:,:,3]=(imnp_raw[:,:,3]*(255.0/imnp_raw[:,:,3].max())).astype(np.uint8)


    imface = Image.new('RGBA', (imw,imh), (0,0,0,0))
    d2 = ImageDraw.Draw(imface)
    d2.text(((imw-w)//2,(imh-h)//2), joined, font=fnt, fill=colors[font_c], stroke_width=7, stroke_fill=colors[outline_c])
    imfacenp = np.asarray(imface)

    blank = np.zeros((imh,imw,4),dtype=np.uint8)
    compnp=np.where((imnp[:,:,3]>0)[:,:,None], imnp, blank)
    compnp=np.where((imfacenp[:,:,3]>200)[:,:,None], imfacenp, compnp)

    Image.fromarray(compnp,'RGBA').save(name)

In [2]:
%cd /content/subtitles
!zip -r subtitles.zip *
files.download('/content/subtitles/subtitles.zip')


/content/subtitles
  adding: a_0000_0000_0003.png (deflated 3%)
  adding: a_0001_0003_0008.png (deflated 3%)
  adding: a_0002_0008_0011.png (deflated 4%)
  adding: a_0003_0011_0013.png (deflated 7%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>